# pLTV Python Model

#### Resources

Github repo
https://github.com/kliao-tala/pLTV

Fader & Hardie paper on sBG model
https://drive.google.com/file/d/1tfMiERon1HgWo8dDJddwSzwc_AXK7LCA/view?usp=sharing
https://faculty.wharton.upenn.edu/wp-content/uploads/2012/04/Fader_hardie_jim_07.pdf

YT tutorial to pull look data
https://www.youtube.com/watch?v=EKwtLBnwXHk&list=PLXwS3L4W3KR2fhnQa-sLyajPZ9-L00KGX&index=1

Looker Python SDK & API
https://inventure.looker.com/extensions/marketplace_extension_api_explorer::api-explorer/4.0

---

In [71]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

from scipy.optimize import curve_fit, minimize

import plotly
from plotly import graph_objects as go
import plotly.io as pio

# change default plotly theme
pio.templates.default = "plotly_white"

In [72]:
inputs = pd.read_csv('data/pltv_inputs.csv')
data = pd.read_csv('data/ke_data.csv')

## Data Munging

In [73]:
# fix date inconsistencies
data = data.replace({'2021-9': '2021-09', '2021-8': '2021-08', '2021-7': '2021-07', 
              '2021-6': '2021-06', '2021-5': '2021-05', '2021-4': '2021-04', '2020-9': '2020-09'})

# sort by months since first disbursement
data = data.sort_values(['First Loan Local Disbursement Month', 
                         'Months Since First Loan Disbursed'])

# remove all columns calculated through looker
data = data.loc[:,:"Default Rate Amount 51D"]

In [74]:
data.head()

,First Loan Local Disbursement Month,Months Since First Loan Disbursed,Count First Loans,Count Borrowers,Count Loans,Total Amount,Total Interest Assessed,Total Rollover Charged,Total Rollover Reversed,Default Rate Amount 7D,Default Rate Amount 30D,Default Rate Amount 51D
0,2020-09,0,7801,7801,13156,48361000,6540240,681325,81520,0.155382,0.121192,0.113031
15,2020-09,1,0,4481,5697,34490000,4660880,416544,32387,0.130661,0.101823,0.095738
2,2020-09,2,0,3661,4310,31461000,4297310,401077,30617,0.139719,0.103958,0.094792
9,2020-09,3,0,3050,3599,30482000,4178400,343062,17629,0.125111,0.089089,0.084399
10,2020-09,4,0,2549,2985,29303000,3964590,300262,920,0.113720,0.086750,0.081658


## pLTV Model

In [75]:
# KSH to USD conversion factor
ksh_usd = 0.00925

In [83]:
class Model:
    """
    sBG model class containing all functionality for creating, analyzing, and backtesting
    the sBG model.
    """
    def __init__(self, data):
        self.data = data

        self.clean_data()
        
    def clean_data(self):
        # fix date inconsistencies
        self.data = self.data.replace({'2021-9': '2021-09', '2021-8': '2021-08', \
                                       '2021-7': '2021-07', '2021-6': '2021-06', \
                                       '2021-5': '2021-05', '2021-4': '2021-04', \
                                       '2020-9': '2020-09'})

        # sort by months since first disbursement
        self.data = self.data.sort_values(['First Loan Local Disbursement Month', 
                                 'Months Since First Loan Disbursed'])

        # remove all columns calculated through looker
        self.data = self.data.loc[:,:"Default Rate Amount 51D"]
        
        # add more convenient cohort column
        self.data['cohort'] = self.data['First Loan Local Disbursement Month']
        
        
    # --- DATA FUNCTIONS --- #
    def borrower_retention(self, cohort_data):
        return 100*cohort_data['Count Borrowers']/cohort_data['Count Borrowers'].max()

    def borrower_survival(self, cohort_data):
        return 100*cohort_data['Count Borrowers']/cohort_data['Count Borrowers'].shift(1)
    
    def loans_per_borrower(self, cohort_data):
        return cohort_data['Count Loans']/cohort_data['Count Borrowers']
    
    def loan_size(self, cohort_data, to_usd):
        df = cohort_data['Total Amount']/cohort_data['Count Loans']
        if to_usd:
            df *= ksh_usd
        return df
    
    def interest_rate(self, cohort_data):
        return 100*cohort_data['Total Interest Assessed']/cohort_data['Total Amount']
    
    def default_rate(self, cohort_data, period=7):
        if period==7:
            return 100*cohort_data['Default Rate Amount 7D'].fillna(0)
        
        elif period==51:
            default_rate = 100*cohort_data['Default Rate Amount 51D']

            recovery_rate_51 = float(inputs[inputs.market=='ke']['recovery_7-30'] + \
                                     inputs[inputs.market=='ke']['recovery_30-51'])

            ## fill null 51dpd values with 7dpd values based on recovery rates
            derived_51dpd = 100*(cohort_data['Count Loans']*(cohort_data['default_rate_7dpd']/100) - \
                cohort_data['Count Loans']*(cohort_data['default_rate_7dpd']/100)*recovery_rate_51)/ \
                cohort_data['Count Loans']
            
            return default_rate.fillna(derived_51dpd)
        
        elif period==365:
            # get actual data if it exists
            default_rate = np.nan*cohort_data['Default Rate Amount 51D']

            recovery_rate_365 = float(inputs[inputs.market=='ke']['recovery_51_'])

            ## fill null 365dpd values with 51dpd values based on recovery rates
            derived_365dpd = 100*(cohort_data['Count Loans']*(cohort_data['default_rate_51dpd']/100) - \
                cohort_data['Count Loans']*(cohort_data['default_rate_51dpd']/100)* \
                recovery_rate_365)/cohort_data['Count Loans']

            return default_rate.fillna(derived_365dpd)
        
    def loans_per_original(self, cohort_data):
        return cohort_data['Count Loans']/cohort_data['Count Borrowers'].max()
    
    def origination_per_original(self, cohort_data, to_usd):
        df = cohort_data['Total Amount']/cohort_data['Count Borrowers'].max()
        if to_usd:
            df *= ksh_usd
        return df
    
    def revenue_per_original(self, cohort_data, to_usd):
        df = (cohort_data['Total Interest Assessed'] + \
                                                 cohort_data['Total Rollover Charged'] - \
                                                 cohort_data['Total Rollover Reversed'])/ \
                                                cohort_data['Count Borrowers'].max()
        if to_usd:
            df *= ksh_usd
        return df
    
    def credit_margin(self, cohort_data):
        return cohort_data['revenue_per_original'] - \
                (cohort_data['origination_per_original'] + cohort_data['revenue_per_original'])* \
                cohort_data['default_rate_365dpd']/100
    
    def opex_per_original(self, cohort_data):
        opex_cost_per_loan = float(inputs[inputs.market=='ke']['opex cost per loan'])
        cost_of_capital = float(inputs[inputs.market=='ke']['cost of capital'])/12
        
        return opex_cost_per_loan*cohort_data['loans_per_original'] + cost_of_capital*cohort_data['origination_per_original']
        
    def generate_features(self, to_usd=True):
        """
        Generate all features required for pLTV model.
        """
        cohorts = []

        # for each cohort
        for cohort in self.data.loc[:,'First Loan Local Disbursement Month'].unique():
            cohort_data = self.data[self.data['First Loan Local Disbursement Month']==cohort].iloc[:-2,:]

            # data functions
            cohort_data['borrower_retention'] = self.borrower_retention(cohort_data)
            cohort_data['borrower_survival'] = self.borrower_survival(cohort_data)
            cohort_data['loans_per_borrower'] = self.loans_per_borrower(cohort_data)
            cohort_data['loan_size'] = self.loan_size(cohort_data, to_usd)
            cohort_data['interest_rate'] = self.interest_rate(cohort_data)
            cohort_data['default_rate_7dpd'] = self.default_rate(cohort_data, period=7)
            cohort_data['default_rate_51dpd'] = self.default_rate(cohort_data, period=51)
            cohort_data['default_rate_365dpd'] = self.default_rate(cohort_data, period=365)
            cohort_data['loans_per_original'] = self.loans_per_original(cohort_data)
            cohort_data['origination_per_original'] = self.origination_per_original(cohort_data, to_usd)
            cohort_data['revenue_per_original'] = self.revenue_per_original(cohort_data, to_usd)
            cohort_data['CM$_per_original'] = self.credit_margin(cohort_data)
            cohort_data['opex_per_original'] = self.opex_per_original(cohort_data)
            cohort_data['LTV_per_original'] = cohort_data['CM$_per_original'] - cohort_data['opex_per_original']
            cohort_data['CM%_per_original'] = 100*cohort_data['LTV_per_original']/cohort_data['revenue_per_original']
            
            # reset the index and append the data
            cohorts.append(cohort_data.reset_index(drop=True))

        self.cohorts = cohorts
        self.data = pd.concat(cohorts, axis=0)
    
    def plot_cohorts(self, param):
        """
        Generate scatter plot for a specific paramter.
        """
        # for each cohort
        curves = []
        for cohort in self.data.loc[:,'First Loan Local Disbursement Month'].unique():
            output = self.data[self.data['First Loan Local Disbursement Month']==cohort]

            output = output[param].reset_index(drop=True)
            output.name = cohort

            curves.append(output)
            
        traces = []

        for cohort in curves:
            traces.append(go.Scatter(name=cohort.name, x=cohort.index, y=cohort))

        fig = go.Figure(traces)
        fig.update_layout(xaxis=dict(title='Month Since Disbursement'),
                         yaxis=dict(title=param))

        fig.show()
        

Review calculations for revenue per original, they're very off. Used a different formula than Liang's. Understand his formula.

#### Issues to fix

1. null values in default rates.
2. forecasting vs not, the final incomplete month.

### Run Model

In [84]:
# create a model object
m = Model(data)

# generate features
m.generate_features()

m.data.head()

,First Loan Local Disbursement Month,Months Since First Loan Disbursed,Count First Loans,Count Borrowers,Count Loans,Total Amount,Total Interest Assessed,Total Rollover Charged,Total Rollover Reversed,Default Rate Amount 7D,...,default_rate_7dpd,default_rate_51dpd,default_rate_365dpd,loans_per_original,origination_per_original,revenue_per_original,CM$_per_original,opex_per_original,LTV_per_original,CM%_per_original
0,2020-09,0,7801,7801,13156,48361000,6540240,681325,81520,0.155382,...,15.538171,11.303088,9.946717,1.686450,57.343834,8.466276,1.920330,2.847339,-0.927009,-10.949436
1,2020-09,1,0,4481,5697,34490000,4660880,416544,32387,0.130661,...,13.066055,9.573774,8.424921,0.730291,40.896359,5.982130,2.032654,1.407028,0.625626,10.458248
2,2020-09,2,0,3661,4310,31461000,4297310,401077,30617,0.139719,...,13.971936,9.479154,8.341656,0.552493,37.304737,5.534787,1.961261,1.133426,0.827836,14.956956
3,2020-09,3,0,3050,3599,30482000,4178400,343062,17629,0.125111,...,12.511146,8.439905,7.427117,0.461351,36.143892,5.340399,2.259313,1.000542,1.258770,23.570717
4,2020-09,4,0,2549,2985,29303000,3964590,300262,920,0.113720,...,11.372050,8.165786,7.185892,0.382643,34.745898,5.055938,2.195821,0.881503,1.314318,25.995533


In [85]:
m.data.columns

Index(['First Loan Local Disbursement Month',
       'Months Since First Loan Disbursed', 'Count First Loans',
       'Count Borrowers', 'Count Loans', 'Total Amount',
       'Total Interest Assessed', 'Total Rollover Charged',
       'Total Rollover Reversed', 'Default Rate Amount 7D',
       'Default Rate Amount 30D', 'Default Rate Amount 51D', 'cohort',
       'borrower_retention', 'borrower_survival', 'loans_per_borrower',
       'loan_size', 'interest_rate', 'default_rate_7dpd', 'default_rate_51dpd',
       'default_rate_365dpd', 'loans_per_original', 'origination_per_original',
       'revenue_per_original', 'CM$_per_original', 'opex_per_original',
       'LTV_per_original', 'CM%_per_original'],
      dtype='object')

In [86]:
m.data[m.data.cohort=='2021-01'].iloc[:,:].revenue_per_original

0     7.955677
1     5.356359
2     5.529705
3     5.117154
4     5.194148
5     4.844846
6     4.999410
7     4.869380
8     4.522021
9     4.550390
10    4.501288
11    4.221572
12    3.531115
Name: revenue_per_original, dtype: float64

In [87]:
m.data[m.data.cohort=='2021-01']

,First Loan Local Disbursement Month,Months Since First Loan Disbursed,Count First Loans,Count Borrowers,Count Loans,Total Amount,Total Interest Assessed,Total Rollover Charged,Total Rollover Reversed,Default Rate Amount 7D,...,default_rate_7dpd,default_rate_51dpd,default_rate_365dpd,loans_per_original,origination_per_original,revenue_per_original,CM$_per_original,opex_per_original,LTV_per_original,CM%_per_original
0,2021-01,0,10295,10295,17510,59816000,8060230,804693,10469,0.148613,...,14.861348,11.569712,10.181346,1.700826,53.744342,7.955677,1.673785,2.827320,-1.153536,-14.499526
1,2021-01,1,0,5359,6578,39655000,5367790,593782,89,0.166274,...,16.627403,13.072764,11.504033,0.638951,35.629796,5.356359,0.641298,1.229405,-0.588106,-10.979589
2,2021-01,2,0,4436,5384,40592000,5560420,593992,0,0.161803,...,16.180338,12.580258,11.070627,0.522972,36.471685,5.529705,0.879888,1.085433,-0.205546,-3.717118
3,2021-01,3,0,3494,4078,37805000,5240160,455094,0,0.133220,...,13.321992,10.526195,9.263052,0.396115,33.967581,5.117154,1.496715,0.890853,0.605861,11.839809
4,2021-01,4,0,3044,3562,38703000,5357470,423476,0,0.120264,...,12.026382,9.479085,8.341595,0.345993,34.774429,5.194148,1.860131,0.833434,1.026697,19.766419
5,2021-01,5,0,2552,2947,36979000,5036530,355653,0,0.107270,...,10.726961,7.511322,6.609963,0.286255,33.225425,4.844846,2.328415,0.737799,1.590616,32.831093
6,2021-01,6,0,2361,2711,37805000,5167550,396658,0,0.116691,...,11.669108,8.543680,7.518439,0.263332,33.967581,4.999410,2.069700,0.715580,1.354120,27.085607
7,2021-01,7,0,2088,2420,36769000,5051170,368318,0,0.110418,...,11.041810,7.190642,6.327765,0.235066,33.036741,4.869380,2.470770,0.668185,1.802585,37.018781
8,2021-01,8,0,1895,2120,34097000,4693110,339777,0,0.111147,...,11.114664,7.977148,7.019890,0.205925,30.635964,4.522021,2.053969,0.603711,1.450258,32.071016
9,2021-01,9,0,1775,2031,34836000,4734720,329741,0,0.105323,...,10.532315,7.304910,6.428321,0.197280,31.299951,4.550390,2.245815,0.599493,1.646322,36.179803


In [88]:
# visualize cohorts for a given feature
m.plot_cohorts('borrower_retention')

In [89]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 181 entries, 0 to 178
Data columns (total 12 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   First Loan Local Disbursement Month  181 non-null    object 
 1   Months Since First Loan Disbursed    181 non-null    int64  
 2   Count First Loans                    181 non-null    int64  
 3   Count Borrowers                      181 non-null    int64  
 4   Count Loans                          181 non-null    int64  
 5   Total Amount                         181 non-null    int64  
 6   Total Interest Assessed              181 non-null    int64  
 7   Total Rollover Charged               181 non-null    int64  
 8   Total Rollover Reversed              181 non-null    int64  
 9   Default Rate Amount 7D               158 non-null    float64
 10  Default Rate Amount 30D              148 non-null    float64
 11  Default Rate Amount 51D         

In [90]:
m.data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149 entries, 0 to 1
Data columns (total 28 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   First Loan Local Disbursement Month  149 non-null    object 
 1   Months Since First Loan Disbursed    149 non-null    int64  
 2   Count First Loans                    149 non-null    int64  
 3   Count Borrowers                      149 non-null    int64  
 4   Count Loans                          149 non-null    int64  
 5   Total Amount                         149 non-null    int64  
 6   Total Interest Assessed              149 non-null    int64  
 7   Total Rollover Charged               149 non-null    int64  
 8   Total Rollover Reversed              149 non-null    int64  
 9   Default Rate Amount 7D               149 non-null    float64
 10  Default Rate Amount 30D              146 non-null    float64
 11  Default Rate Amount 51D           

## Forecasting

### Power law regression

In [91]:
def power_law(x, A, B):
    return A*x**B

In [98]:
arr = m.data[m.data['First Loan Local Disbursement Month']=='2020-09'].loc[1:, 'borrower_retention']
arr = arr.dropna()
power_param, power_cov = curve_fit(power_law, arr.index, arr)

In [99]:
x = list(range(1,25))
power_fit = power_law(x, power_param[0], power_param[1])

In [100]:
traces = [
    go.Scatter(name='actual', x=arr.index, y=arr),
    go.Scatter(name='power-law', x=x, y=power_fit)
]

fig = go.Figure(traces)

fig.show()

### sBG probalistic model

sBG model assumptions:
1. The propensity of one customer to drop out is independent of the behavior of every other customer.
2. Individual customer retention rates are unchanged over time.
3. Observed retention increase with time due to aggregate results and heterogenity in customer behaviors.
4. Model applies for customer relationships in "discrete-time" and "contractual" settings.

In [101]:
# initial guesses @ alpha and beta
alpha = 1
beta = 1

In [103]:
def p(t, alpha, beta):
    if t==1:
        return alpha/(alpha + beta)
    else:
        return p(t-1, alpha, beta) * (beta+t-2)/(alpha+beta+t-1)
    
def s(t, alpha, beta):
    if t==1:
        return 1 - p(t, alpha, beta)
    else:
        return s(t-1, alpha, beta) - p(t, alpha, beta)
    
def log_likelihood(params):
    alpha, beta = params
    ll=0
    for t in c[1:].index:
        ll += (c[t-1]-c[t])*np.log(p(t, alpha, beta))
    ll += c.iloc[-1]*np.log(s((len(c)-1)-1, alpha, beta)-p(len(c)-1, alpha, beta))
    return -ll

In [111]:
cohort = '2020-09'

In [112]:
c = m.data[m.data['First Loan Local Disbursement Month']==cohort]['Count Borrowers']
c = c.reset_index(drop=True)

In [113]:
# since we're working with logs, we need bounds for alpha and beta > 0.
bounds = ((0,None), (0,None))

results = minimize(log_likelihood, np.array([1,1]), bounds=bounds)
results

      fun: 15970.457182334316
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.0001819, -0.0001819])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 36
      nit: 9
     njev: 12
   status: 0
  success: True
        x: array([0.71002432, 1.04180686])

In [114]:
alpha_opt, beta_opt = results.x

sBG_forecast = []
for i in x:
    sBG_forecast.append(100*s(i, alpha_opt, beta_opt))

In [115]:
arr = m.data[m.data['First Loan Local Disbursement Month']==cohort].loc[1:, 'borrower_retention']
arr = arr.dropna()

power_param, power_cov = curve_fit(power_law, arr.index, arr)
power_fit = power_law(x, power_param[0], power_param[1])

In [116]:
traces = [
    go.Scatter(name='actual', x=arr.index, y=arr),
    go.Scatter(name='power-law', x=x, y=power_fit),
    go.Scatter(name='sBG', x=x, y=sBG_forecast)
]

fig = go.Figure(traces)
fig.update_layout(xaxis=dict(title='Months Since First Loan'),
                 yaxis=dict(title='Retention (%)'))

fig.show()

In [117]:
power_res = abs(power_fit[:len(arr)] - arr)
sbg_res = abs(sBG_forecast[:len(arr)] - arr)

traces = [
    go.Scatter(name='power-law', x=arr.index, y=power_res),
    go.Scatter(name='sBG', x=arr.index, y=sbg_res)
]

fig = go.Figure(traces)
fig.update_layout(title='Model Residuals', xaxis=dict(title='Months Since First Loan'))

fig.show()

#### Scale forecast

In [118]:
forecast_dfs = []

alpha = beta = 1

x = list(range(1, 25))
df = m.data[['cohort', 'Count Borrowers']]
for cohort in df.cohort.unique():
    c_data = df[df.cohort==cohort]
    n = c_data.loc[0, 'Count Borrowers']
    
    if len(c_data) > 3:
        c = c_data['Count Borrowers']

        # fit model
        bounds = ((0,None), (0,None))
        results = minimize(log_likelihood, np.array([1,1]), bounds=bounds)

        # forecast
        forecast = []
        for i in x:
            forecast.append(n*s(i, results.x[0], results.x[1]))

        forecast = pd.DataFrame(forecast, index=x, columns=['Count Borrowers'])
    
        holder_df = pd.DataFrame(np.nan, index=range(0,25), columns=['null'])
        
        c_data['data_type'] = 'actual'
        
        c_data = pd.concat([c_data, holder_df], axis=1).drop('null', axis=1)
        c_data.cohort = c_data.cohort.ffill()

        c_data.data_type = c_data.data_type.fillna('forecast')
        
        forecast_dfs.append(c_data.fillna(forecast))
        
forecast = pd.concat(forecast_dfs)

In [127]:
cohort='2021-06'

df = forecast[forecast.cohort==cohort]
#df['retention'] = df['Count Borrowers']/df['Count Borrowers'].max()

traces = []
for dtype in df.data_type.unique():
    traces.append(go.Scatter(name=dtype, x=df[df.data_type==dtype].index, 
                             y=df[df.data_type==dtype]['Count Borrowers'], mode='markers+lines'))
    
fig = go.Figure(traces)
fig.update_layout(xaxis=dict(title='Months Since Disbursement'),
                 yaxis=dict(title='Count Borrowers'))

fig.show()

### Backtest Framework

In [120]:
def test(actual, forecast, metric='rmse'):
    """
    Test forecast performance against actuals using method defined by metric.
    """
    if metric=='rmse':
        error = np.sqrt(sum((forecast[:len(actual)] - actual)**2))
    elif metric=='mae':
        error = np.mean(forecast[:len(actual)] - actual)
    return error

In [121]:
test(arr, power_fit, metric='rmse') 

9.916461969436115

In [122]:
test(arr, sBG_forecast)

5.942824530450512

### Test all cohorts

In [123]:
test_vals = {}

for cohort in m.data.cohort.unique():
    x = list(range(1,25))

    # power-law
    arr = m.data[m.data.cohort==cohort]['borrower_retention'][1:].dropna()
    
    if len(arr)>=3:
        power_param, power_cov = curve_fit(power_law, arr.index, arr)
        power_fit = power_law(x, power_param[0], power_param[1])

        # sbg
        c = m.data[m.data.cohort==cohort]['Count Borrowers'].reset_index(drop=True)

        # since we're working with logs, we need bounds for alpha and beta > 0.
        bounds = ((0,None), (0,None))

        results = minimize(log_likelihood, np.array([1,1]), bounds=bounds)

        sBG_forecast = []
        for i in x:
            sBG_forecast.append(100*s(i, results.x[0], results.x[1]))

        test_vals[cohort] = {'power-law': test(arr, power_fit, metric='rmse'), 
                             'sbg': test(arr, sBG_forecast)}
    
test_vals

{'2020-09': {'power-law': 9.916461969436115, 'sbg': 5.942824530450512},
 '2020-10': {'power-law': 8.878383231455054, 'sbg': 6.802539463007343},
 '2020-11': {'power-law': 7.41916331953283, 'sbg': 5.531285348505579},
 '2020-12': {'power-law': 6.790207358997688, 'sbg': 5.201175246265757},
 '2021-01': {'power-law': 7.911571109795891, 'sbg': 6.382120180468375},
 '2021-02': {'power-law': 5.44267072441055, 'sbg': 3.835302216820124},
 '2021-03': {'power-law': 7.761078473343302, 'sbg': 6.929119754451952},
 '2021-04': {'power-law': 5.722561737841632, 'sbg': 4.604031032253486},
 '2021-05': {'power-law': 6.2827668657365425, 'sbg': 5.168228245303411},
 '2021-06': {'power-law': 4.683240567322109, 'sbg': 3.9029195927544857},
 '2021-07': {'power-law': 4.955487707285651, 'sbg': 4.4694318959938215},
 '2021-08': {'power-law': 4.515225858421871, 'sbg': 4.553677985892246},
 '2021-09': {'power-law': 1.9865115724016997, 'sbg': 1.7385765475413657},
 '2021-10': {'power-law': 0.1274764675060846, 'sbg': 0.278399

In [ ]:
m.data[m.data.cohort=='2021-10']

In [ ]:
forecast_methods = {
    'Count Borrowers': 'sbg',
    'loans_per_borrwer': 'power-law'
}

In [ ]:
m.data[m.data['First Loan Local Disbursement Month']=='2020-09']['Count Borrowers']

#### Questions/Concerns

1. Is there a framework that's been developed/used to back test this forecast model?
2. Filter out bad cohorts. Why are some starting at 0? (e.g. 2021-07)
3. Why are the last few months not included? Why not just omit the final incomplete month?


In [ ]:
new_data[new_data['First Loan Local Disbursement Month'] == '2021-07']